#  Text Classification with Tensorflow Keras API
Example based on Tensorflow's text-classification tutorial. [Here](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/basic_text_classification.ipynb) is the original tutorial notebook.

In [1]:
from __future__ import absolute_import, division, print_function
import sys
import os

import tensorflow as tf
from tensorflow import keras

import numpy as np
print(tf.__version__)

# Import local bentoml repository
sys.path.insert(0, os.path.join(os.getcwd(), "../../.."))
import bentoml
print(bentoml.__file__)

1.13.1
/Users/bozhaoyu/src/bento/examples/aws-lambda/model/../../../bentoml/__init__.py


## Download the IMDB dataset

In [2]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [3]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(encoded_text):
    return ' '.join([reverse_word_index.get(i, '?') for i in encoded_text])

In [4]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

## Build the model

In [5]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

## Create a validation set

In [7]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## Train the model

In [8]:
model.fit(partial_x_train,
                    partial_y_train,
                    epochs=10,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
15000/15000 [==============================] - 1s 36us/sample - loss: 0.6918 - acc: 0.5933 - val_loss: 0.6901 - val_acc: 0.6609
Epoch 2/10
15000/15000 [==============================] - 0s 22us/sample - loss: 0.6868 - acc: 0.7160 - val_loss: 0.6831 - val_acc: 0.7349
Epoch 3/10
15000/15000 [==============================] - 0s 21us/sample - loss: 0.6755 - acc: 0.7487 - val_loss: 0.6682 - val_acc: 0.7430
Epoch 4/10
15000/15000 [==============================] - 0s 23us/sample - loss: 0.6544 - acc: 0.7570 - val_loss: 0.6435 - val_acc: 0.7474
Epoch 5/10
15000/15000 [==============================] - 0s 24us/sample - loss: 0.6220 - acc: 0.7851 - val_loss: 0.6090 - val_acc: 0.7759
Epoch 6/10
15000/15000 [==============================] - 0s 23us/sample - loss: 0.5799 - acc: 0.8068 - val_loss: 0.5679 - val_acc: 0.7988
Epoch 7/10
15000/15000 [==============================] - 0s 21us/sa

## Evaluate the model

In [12]:
results = model.evaluate(test_data, test_labels)

print(results)

25000/25000 [==============================] - 0s 15us/sample - loss: 0.4190 - acc: 0.8430
[0.4189959478187561, 0.84304]


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [14]:
%%writefile text_classification_service.py
import pandas as pd
import numpy as np
from tensorflow import keras
from bentoml import api, env, BentoService, artifacts
from bentoml.artifact import TfKerasModelArtifact, PickleArtifact
from bentoml.handlers import JsonHandler

@artifacts([
    TfKerasModelArtifact('model'),
    PickleArtifact('word_index')
])
@env(requirements_txt="""tensorflow>=1.13.1 
numpy>=1.16.2
pandas>=0.24.2
""")
class TextClassificationService(BentoService):
   
    def word_to_index(self, word):
        if word in self.artifacts.word_index:
            return self.artifacts.word_index[word]
        else:
            return self.artifacts.word_index["<UNK>"]
    
    @api(JsonHandler)
    def predict(self, parsed_json):
        """
        """
        text = parsed_json['text']
        
        sequence = keras.preprocessing.text.hashing_trick(
            text,
            256,
            hash_function=self.word_to_index,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            lower=True,
            split=' ')

        return self.artifacts.model.predict(np.expand_dims(sequence, 0))

Overwriting text_classification_service.py


In [15]:
from text_classification_service import TextClassificationService

svc = TextClassificationService.pack(model=model, word_index=word_index)
saved_path = svc.save('../serverless/vendors', version="v1_0_0")
print(saved_path)

ValueError: Version v1_0_0 in Path: ../serverless/vendors already exist.

In [16]:
svc.predict({ 'text': 'the best movie I have ever seen' })

array([[0.9898762]], dtype=float32)

In [24]:
result = svc.predict({ 'text': 'This is a bad movie' })
print(result.tolist())

[[0.10192883759737015]]
